### Part 1 : Add new vocabulary to Tokenizer

In [1]:
import transformers
print(transformers.__version__)
from transformers.utils import send_example_telemetry
from transformers import BertModel, BertTokenizer, BertForMaskedLM , AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
model_checkpoint  = r"D:\Projects\Huggingface\bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = BertForMaskedLM.from_pretrained(model_checkpoint)
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)
import numpy as np
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
nlp = spacy.load("en_core_web_sm")
import json

d:\Projects\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.37.1


d:\Projects\.venv\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at D:\Projects\Huggingface\bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identica

cuda


In [2]:
corpus_file = pd.read_excel(r"C:\Users\Mithun\Downloads\cleancorpus_5.xlsx")
corpus_file = corpus_file[~corpus_file['text'].isna()]
replacer = {'\n':'',"[\[].*?[\]]": "",'[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’""′‘\\\]':"", ' +': ' '}
corpus_file['text'] = corpus_file['text'].replace(replacer, regex=True)
corpus_file['text'] = corpus_file['text'].apply(lambda x: x.strip())
documents = corpus_file['text']
cat = corpus_file['Category']

In [3]:
def spacy_tokenizer(document, nlp=nlp):
    # tokenize the document with spaCY
    doc = nlp(document)
    # Remove stop words and punctuation symbols
    tokens = [token.text for token in doc]
    return tokens

tfidf_vectorizer = TfidfVectorizer(lowercase=False, tokenizer=spacy_tokenizer, 
    norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)
# parse matrix of tfidf
length = len(documents)
result = tfidf_vectorizer.fit_transform(documents)


d:\Projects\.venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [4]:
# get idf of tokens
idf = tfidf_vectorizer.idf_
# get tokens from most frequent in documents to least frequent
idf_sorted_indexes = sorted(range(len(idf)), key=lambda k: idf[k])
idf_sorted = idf[idf_sorted_indexes]
tokens_by_df = np.array(tfidf_vectorizer.get_feature_names_out())[idf_sorted_indexes]

In [5]:
length
#len(tokens_by_df)

422752

In [6]:
def dfreq(idf, N):
    return (1+N) / np.exp(idf - 1) - 1

dfreqs_sorted = dfreq(idf_sorted, length).astype(np.int32)
tokens_dfreqs = {tok:dfreq for tok, dfreq in zip(tokens_by_df,dfreqs_sorted)}
tokens_pct_list = [int(round(dfreq/length*100,2)) for token,dfreq in tokens_dfreqs.items()]
# choose the proportion of new tokens to add in vocabulary
pct = 1 # all tokens present in at least 1%
index_max = 200000#len(np.array(tokens_pct_list)[np.array(tokens_pct_list)>=pct])
new_tokens = tokens_by_df[:index_max]

In [18]:
with open("newtoken.txt", 'r') as f:
    new_tokens = json.load(f)

In [7]:
added_tokens = tokenizer.add_tokens(new_tokens)
model.resize_token_embeddings(len(tokenizer))

Embedding(216841, 768)

### Part 2 : Tune the model on the new corpus with the enhanced Tokenizer

In [8]:
send_example_telemetry("language_modeling_notebook", framework="pytorch")

In [9]:
from datasets import load_dataset
datasets = load_dataset('csv', data_files = {'train': r"C:\Users\Mithun\Downloads\train.csv", 'test': r"C:\Users\Mithun\Downloads\test.csv"})

In [10]:
def tokenize_function(examples, tokenizer=tokenizer):
    return tokenizer(examples["text"])

In [11]:
tokenize_function({'text':'Test sentence for the tokenizer'})

{'input_ids': [101, 3231, 30537, 6251, 30537, 2005, 30537, 1996, 30537, 166551, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
block_size = 128

def group_texts(examples, block_size=block_size):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4): 100%|██████████| 42276/42276 [00:08<00:00, 4877.88 examples/s]


In [13]:
lm_datasets.keys()

dict_keys(['train', 'test'])

In [15]:
model_name = 'bert_base_itsm'
training_args = TrainingArguments(
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    output_dir=r'D:/Projects/bert_base_itsm-finetuned',
    save_total_limit=5,
    save_strategy="no",
    load_best_model_at_end=False
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
)

In [16]:
trainer.train()

  0%|          | 500/218250 [01:54<13:32:54,  4.46it/s]

{'loss': 2.9025, 'learning_rate': 1.9954180985108823e-05, 'epoch': 0.01}


  0%|          | 1000/218250 [04:05<13:35:03,  4.44it/s]

{'loss': 2.3332, 'learning_rate': 1.990836197021764e-05, 'epoch': 0.01}


  1%|          | 1500/218250 [06:17<13:44:53,  4.38it/s] 

...

{'loss': 1.9725, 'learning_rate': 1.9541809851088203e-05, 'epoch': 0.07}


  3%|▎         | 5500/218250 [23:40<13:22:38,  4.42it/s] 

{'loss': 1.9461, 'learning_rate': 1.9495990836197025e-05, 'epoch': 0.08}


  3%|▎         | 5614/218250 [24:22<13:40:51,  4.32it/s] 

KeyboardInterrupt: 

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")